In [1]:
import random
import string
import numpy as np
import pandas as pd
from tabulate import tabulate

### Config

In [43]:
DISH_COUNT = 300
MIN_CALORIES_PER_DISH = 100
MAX_CALORIES_PER_DISH = 1200

DAYS_PER_MENU = 7
DISHES_PER_DAY = 3
PENALTY = 400

POPULATION_SIZE = 20
MATING_POPULATION_SIZE = 10
ITERATIONS = 3000

CALORIES_PER_DAY = 2000
REAL_DATA = True

#### Helper functions

In [21]:
def visualize_menu(menu):
    print("Score: " + str(score_menu(menu)))
    headers = ["Day", "Calories", "Dishes"]
    calories = count_calories_per_day(menu)
    table = []
    for day in range(DAYS_PER_MENU):
        table.append([day, calories[day], menu[day]])
    print(tabulate(table, headers, tablefmt="fancy_grid"))

#### Test data

In [22]:
if not REAL_DATA:
    dishes = {}
    for _ in range(DISH_COUNT):
        name = ''.join(random.choices(string.ascii_lowercase, k=6))
        dishes[name] = random.randint(MIN_CALORIES_PER_DISH, MAX_CALORIES_PER_DISH)
    dishes = list(dishes.items())

    print(*dishes[:7], sep='n')

#### Real data

In [23]:
if REAL_DATA:
    dishes = {}
    data = pd.read_csv('data.csv', delimiter=',', encoding='utf-8', low_memory=False)
    df = pd.DataFrame(data, columns=['name', 'Food Group', "calories_portion"])
    
    while len(dishes) != DISH_COUNT:
        random_choice = df.sample()
        if MIN_CALORIES_PER_DISH < random_choice.calories_portion.values[0] < MAX_CALORIES_PER_DISH:
            dishes[random_choice.name.values[0]] = random_choice.calories_portion.values[0]
    dishes = list(dishes.items())
    print(*dishes[:7], sep='\n')

('Lamb New Zealand Imported Square-Cut Shoulder Chops Separable Lean And Fat Cooked Braised', 273.7)
('Jams And Preserves No Sugar (With Sodium Saccharin) Any Flavor', 295.68)
('Beef New Zealand Imported Intermuscular Fat Cooked', 476.0)
('Chimichanga With Meat And Sour Cream', 259.44)
('Pie Mince Individual Size Or Tart', 358.02)
('Baklava', 333.84)
('Pork Fresh Loin Center Loin (Chops) Bone-In Separable Lean And Fat Cooked Braised', 205.7)


#### Sample random menu

In [24]:
def create_random_menu():
    menu = np.ndarray((7, 3), tuple)
    for day in range(DAYS_PER_MENU):
        for dish_index in range(DISHES_PER_DAY):
            menu[day, dish_index] = random.choice(dishes)
    return menu

In [25]:
def count_calories_per_day(menu):

    calories = []
    for day in range(DAYS_PER_MENU):
        calories.append(0)
        for dish_index in range(DISHES_PER_DAY):
            calories[-1] += menu[day, dish_index][1]
    
    return calories

def count_attr_per_day(menu):
    
    names = np.reshape(menu, 21)
    unique, counts = np.unique(names, return_counts=True)
    names_dict = dict(zip(unique, counts))

    attr = []
    for day in range(DAYS_PER_MENU):
        attr.append(0)
        for dish_index in range(DISHES_PER_DAY):
            attr[-1] += (names_dict[menu[day, dish_index]] - 1 ) * PENALTY
    
    return attr

def score_menu(menu):
    
    differences = [calories - CALORIES_PER_DAY for calories in count_calories_per_day(menu)]
    penalty = [attr ** 2 for attr in count_attr_per_day(menu)]
    squares = [calories ** 2 for calories in differences]
    
    return sum(squares + penalty)

In [26]:
sample_menu = create_random_menu()
visualize_menu(sample_menu)

Score: 13883311.196560001
╒═══════╤════════════╤════════════════════════════════════════════════════════════════════════════════════════════════╕
│   Day │   Calories │ Dishes                                                                                         │
╞═══════╪════════════╪════════════════════════════════════════════════════════════════════════════════════════════════╡
│     0 │   1061.4   │ [('Beef New Zealand Imported Intermuscular Fat Cooked', 476.0)                                 │
│       │            │  ('Pasta Whole Grain With Cream Sauce And Added Vegetables Restaurant', 460.0)                 │
│       │            │  ('Litchis', 125.4)]                                                                           │
├───────┼────────────┼────────────────────────────────────────────────────────────────────────────────────────────────┤
│     1 │    787.91  │ [('Dominos 14 Inch Sausage Pizza Ultimate Deep Dish Crust', 357.33)                            │
│       │     

In [27]:
def crossover(first_menu, second_menu):
    child = first_menu.copy()
    for day_index in range(DAYS_PER_MENU):
        if random.choice((0, 1)) == 1:
            child[day_index] = second_menu[day_index]
    return child


def random_mutate(menu):
    menu[random.randint(0, DAYS_PER_MENU - 1), random.randint(0, DISHES_PER_DAY - 1)] = random.choice(dishes)
    return menu


def swap_mutate(menu):
    first_day_index = random.randint(0, DAYS_PER_MENU - 1)
    second_day_index = random.randint(0, DAYS_PER_MENU - 1)
    dish_index = random.randint(0, DISHES_PER_DAY - 1)
    menu[first_day_index, dish_index], menu[second_day_index, dish_index] = menu[second_day_index, dish_index], menu[
        first_day_index, dish_index]
    return menu

In [44]:
population = []
for _ in range(POPULATION_SIZE):
    population.append(create_random_menu())

In [45]:
for i in range(ITERATIONS):
    population.sort(key=score_menu)
    mating_population = population[:MATING_POPULATION_SIZE]

    offspring_population = []
    for _ in range(POPULATION_SIZE - MATING_POPULATION_SIZE):
        offspring_population.append(crossover(random.choice(mating_population), random.choice(mating_population)))
        if random.random() < 0.1:
            offspring_population[-1] = random_mutate(offspring_population[-1])
        if random.random() < 0.1:
            offspring_population[-1] = swap_mutate(offspring_population[-1])

    population = mating_population + offspring_population

In [46]:
def check_duplicant(menu):
    names = np.reshape(menu, 21)
    unique, counts = np.unique(names, return_counts=True)
    is_repetable = dict(zip(unique, counts))
    
    for elem in is_repetable:
        if is_repetable[elem] > 2:
            print("duplicants detected")
            return
    print("No duplicants")

    
population.sort(key=score_menu)
visualize_menu(population[0])    
check_duplicant(population[0])

Score: 321856.84569999995
╒═══════╤════════════╤════════════════════════════════════════════════════════════════════════════════════════════════╕
│   Day │   Calories │ Dishes                                                                                         │
╞═══════╪════════════╪════════════════════════════════════════════════════════════════════════════════════════════════╡
│     0 │    1976.8  │ [('Cheeseburger 1 Large Patty Plain On Bun From Fast Food / Restaurant', 600.0)                │
│       │            │  ('Potatoes Mashed Dehydrated Granules Without Milk Dry Form', 744.0)                          │
│       │            │  ('Trail Mix With Nuts And Fruit', 632.8)]                                                     │
├───────┼────────────┼────────────────────────────────────────────────────────────────────────────────────────────────┤
│     1 │    2002.36 │ [('Hamburger Large Single Patty With Condiments', 437.76)                                      │
│       │     